# FluidDev Test

Conversion from 2D Cartesian Bounding Box to 3D Spherical Bounding Box.

In [1]:
# Imports
import math
from typing import List

In [2]:
# Initialization constants
# Lens constants assuming a 1080p image
f = 714.285714
center = [960, 540]
D = 1.082984  # For image-1, switch to 0.871413 for image-2

In [3]:
# Utility functions
def cartesian2sphere(pt):
    x = (pt[0] - center[0]) / f
    y = (pt[1] - center[1]) / f

    r = math.sqrt(x*x + y*y)
    if r != 0:
        x /= r
        y /= r
    r *= D
    sin_theta = math.sin(r)
    x *= sin_theta
    y *= sin_theta
    z = math.cos(r)

    return [x, y, z]


def sphere2cartesian(pt):
    r = math.acos(pt[2])
    r /= D
    if pt[2] != 1:
        r /= math.sqrt(1 - pt[2] * pt[2])
    x = r * pt[0] * f + center[0]
    y = r * pt[1] * f + center[1]
    return [x, y]


def convert_point(point: List[int]) -> List[int]:
    """Convert single points between Cartesian and spherical coordinate systems"""
    if len(point) == 2:
        return cartesian2sphere(point)
    elif len(point) == 3:
        return sphere2cartesian(point)
    else:
        raise ValueError(f'Expected point to be 2 or 3D, got {len(point)} dimensions')

Using the utility functions, it is possible to convert a list of 2D Cartesian points (x,y) to 3D spherical points, and viceversa. Let's see some examples:

In [4]:
cartesian_point = [100, 200] # (x,y)
spherical_point = [100, 200, 0.5] # (x,y,z)

# Cartesian to spherical
cartesian_to_spherical = convert_point(cartesian_point)
print(f'The point {cartesian_point} in Spherical Coordinates is {cartesian_to_spherical}')

# Spherical to Cartesian
spherical_to_cartesian = convert_point(spherical_point)
print(f'The point {spherical_point} in Cartesian Coordinates is {spherical_to_cartesian}')

The point [100, 200] in Spherical Coordinates is [-0.9167619380064861, -0.3624407661886107, 0.16788162504337606]
The point [100, 200, 0.5] in Cartesian Coordinates is [80713.16187710896, 160046.3237542179]


Now, based on the fact that we can convert from Cartesian to Spherical Coordinates, we wish to be able to convert a Cartesian Bounding Box, to a Spherical Bounding Box.

First, for the sake of simplicity, we are going to define the bounding box as a list containing the coordinates of upper left corner as (xmin, ymin) and lower right corner as (xmax, ymax):

In [5]:
# Simplest bounding box
# Format is xmin ymin xmax ymax
cartesianBox = [100, 50, 200, 100]

Now, when converting from Cartesian to Spherical, the new variable z will vary depending on the (x,y) positions. In order to define a Spherical Bounding Box, since it is 3D, we would need to know the vertices of the cube that encloses the projected 2D bounding box in the spherical surface. But since positions (x,y) are the same for the lower and upper layers of that 3D cube, we can use the upper left and lower right coordinates of the cube to define the 3D Bounding Box, as well as the z coordinate for the upper layer and the lower layer.

So, depending on the coordinates of the cartesian box, we need to get the z values of all vertices of the cartesian box in order to know the lowest and higest values of z in spherical coordinates:

In [6]:
# Given the cartesian box, extract vertices of 2D box
upperLeft = [cartesianBox[0], cartesianBox[1]]
upperRight = [cartesianBox[2], cartesianBox[1]]
lowerLeft = [cartesianBox[0], cartesianBox[3]]
lowerRight = [cartesianBox[2], cartesianBox[3]]

# Store the box vertices
box_vertices = [upperLeft, upperRight, lowerLeft, lowerRight]

# Calculate the spherical coordinates of the vertices
spherical_coordinates = [convert_point(point) for point in box_vertices]

# From the spherical coordinates, extract the lower and higher z values
min_z = min([p[2] for p in spherical_coordinates])
max_z = max([p[2] for p in spherical_coordinates])

Now, to enclose the Cartesian Bounding Box in the Spherical system, I will find the lowest x and y spherical coordinates, and the highest x and y spherical coordinates, in order to define the corner vertices of the final enclosing Spherical Bounding Box:

In [7]:
spherical_coordinates

[[-0.8667310596165098, -0.4938351386187091, 0.0700294663801069],
 [-0.8237449485887304, -0.5310987168532604, 0.19843994716631608],
 [-0.8852387784979313, -0.4529128634175462, 0.10593508953321185],
 [-0.840761193211832, -0.48675648028053425, 0.2370416522342064]]

In [8]:
# From the spherical coordinates, extract the lower and higher x and y values
xmin_spherical = min([p[0] for p in spherical_coordinates])
xmax_spherical = max([p[0] for p in spherical_coordinates])
ymin_spherical = min([p[1] for p in spherical_coordinates])
ymax_spherical = max([p[1] for p in spherical_coordinates])

Finally, the corner vertices of the Spherical Bounding Box will be:

In [9]:
# Format is xmin, ymin, zmin, xmax, ymax, zmax
sphericalBox = [xmin_spherical, ymin_spherical, min_z, xmax_spherical, ymax_spherical, max_z]
sphericalBox

[-0.8852387784979313,
 -0.5310987168532604,
 0.0700294663801069,
 -0.8237449485887304,
 -0.4529128634175462,
 0.2370416522342064]

Let's write a function that condenses the calculation:

In [10]:
def bbox_to_spherical(cartesian):
  # Given the cartesian box, extract vertices of 2D box
  upperLeft = [cartesian[0], cartesian[1]]
  upperRight = [cartesian[2], cartesian[1]]
  lowerLeft = [cartesian[0], cartesian[3]]
  lowerRight = [cartesian[2], cartesian[3]]

  # Store the box vertices
  box_vertices = [upperLeft, upperRight, lowerLeft, lowerRight]

  # Calculate the spherical coordinates of the vertices
  spherical_coordinates = [convert_point(point) for point in box_vertices]

  # From the spherical coordinates, extract the lower and higher z values
  min_z = min([p[2] for p in spherical_coordinates])
  max_z = max([p[2] for p in spherical_coordinates])

  # From the spherical coordinates, extract the lower and higher x and y values
  xmin_spherical = min([p[0] for p in spherical_coordinates])
  xmax_spherical = max([p[0] for p in spherical_coordinates])
  ymin_spherical = min([p[1] for p in spherical_coordinates])
  ymax_spherical = max([p[1] for p in spherical_coordinates])

  # Define spherical Box
  sphericalBox = [xmin_spherical, ymin_spherical, min_z, xmax_spherical, ymax_spherical, max_z]

  return sphericalBox

In [11]:
# Convert Cartesian Box to Spherical Box
mysphericalBox = bbox_to_spherical(cartesianBox)
mysphericalBox

[-0.8852387784979313,
 -0.5310987168532604,
 0.0700294663801069,
 -0.8237449485887304,
 -0.4529128634175462,
 0.2370416522342064]

Now, can we generalize this calculation for any given Polygon in 2D Cartesian space?

Let's say we define a Cartesian Polygon by randomly selecting cartesian points in and adequate interval, and let's asume that we join those points by lines in a way that we get a convex polygon (but it could also be no convex):

In [16]:
# Randomly define vertices points of a Cartesian polygon
import random

Nsides = 10
polygon_vertices = [[random.randint(0, 100), random.randint(0, 150)] for i in range(Nsides)]

In [17]:
polygon_vertices

[[37, 81],
 [4, 67],
 [58, 135],
 [38, 74],
 [67, 104],
 [51, 88],
 [61, 7],
 [40, 44],
 [25, 47],
 [41, 96]]

Now, we need to define a bounding box for that Cartesian Polygon. Following the same logic as before, we can find that bounding box by the min and max values of the coordinates:

In [18]:
# Min and Max coordinates values of Cartesian Polygon
xmin_polygon = min(p[0] for p in polygon_vertices)
xmax_polygon = max(p[0] for p in polygon_vertices)
ymin_polygon = min(p[1] for p in polygon_vertices)
ymax_polygon = max(p[1] for p in polygon_vertices)

Then, the cartesian bounding box is defined by the xmin,ymin and xmax,ymax coordinates:

In [19]:
# Cartesian bounding box of cartesian polygon
polygon_boundingBox = [xmin_polygon, ymin_polygon, xmax_polygon, ymax_polygon]
polygon_boundingBox

[4, 7, 67, 135]

Now that we have the cartesian bounding box, we can use the defined function to obtain the spherical bounding box for the whole Cartesian Polygon:

In [20]:
# Calculate the spherical polygon box
mysphericalPolygonBox = bbox_to_spherical(polygon_boundingBox)
mysphericalPolygonBox

[-0.9207758001409693,
 -0.5125055159212801,
 -0.08860931698635718,
 -0.858663087650475,
 -0.3900776140764566,
 0.08401256335017007]

With this code, we can then proceed to write a script or Classes in order to deal with the problem in an OOP manner. However, I am more familiar with functional programming and data science analysis, and even when I know the theory behind OOP and writing classes, I rather do now complete the spherical_objects.py script as requested since I it would take me more time to refresh concepts and feel confident about writing classes in Python. I would add some comments to the original code that points to the final solution:

In [ ]:
# spherical_objects.py
import math
from typing import List

# Lens constants assuming a 1080p image
f = 714.285714
center = [960, 540]
D = 1.082984  # For image-1, switch to 0.871413 for image-2

def cartesian2sphere(pt):
    x = (pt[0] - center[0]) / f
    y = (pt[1] - center[1]) / f

    r = math.sqrt(x*x + y*y)
    if r != 0:
        x /= r
        y /= r
    r *= D
    sin_theta = math.sin(r)
    x *= sin_theta
    y *= sin_theta
    z = math.cos(r)

    return [x, y, z]

def sphere2cartesian(pt):
    r = math.acos(pt[2])
    r /= D
    if pt[2] != 1:
        r /= math.sqrt(1 - pt[2] * pt[2])
    x = r * pt[0] * f + center[0]
    y = r * pt[1] * f + center[1]
    return [x, y]

def convert_point(point: List[int]) -> List[int]:
    """Convert single points between Cartesian and spherical coordinate systems"""
    if len(point) == 2:
        return cartesian2sphere(point)
    elif len(point) == 3:
        return sphere2cartesian(point)
    else:
        raise ValueError(f'Expected point to be 2 or 3D, got {len(point)} dimensions')

class CartesianBbox:

    def __init__(self, points: List[int], fmt: str):
        # Since the constructor deals with different kinds of format for the bbox, all of those
        # formats must be addressed in the code. In my example, only used xyxy format
        # But any other format can be converted to xyxy format and then use it as it is
        assert fmt in ['xyxy', 'xywh', 'cxcywh'], 'Invalid bbox format'
        assert len(points) == 4, 'Cartesian bbox must have 4 values'
        self.points = points
        self.fmt = fmt


class SphericalBbox:

    def __init__(self):
        # In the constructor, we should define the structure of list of points for the spherical bbox
        # In my case, I would use the xyzxyz format for box corners in a similar way
        # than the CartesianBox constructor, only adding the z dimension
        pass  # Question 1


def bbox_to_spherical(cartesian: CartesianBbox) -> SphericalBbox:
    # This code would be similar to the function I wrote, only
    # dealing correctly with the format and points, and the return the
    # spherical box coordinates
    pass  # Question 2


class CartesianPolygon:

    def __init__(self):
        # In this part, given the N as parameter, we could receive the 
        # polygon coordinates as input list, or create a new one
        pass  # Question 3


class SphericalPolygon:

    def __init__(self):
        # Here qe would define the spherical polygon similar to the spherical
        # box since we only store the rectangle that encloses de polygon
        pass  # Question 4


def polygon_to_spherical(cartesian: CartesianPolygon) -> SphericalPolygon:
    # Here I would calculate the coordinates of the enclosing rectangle for the
    # cartesian polygon and then calculate the spherical bounding box for it 
    # as a final result
    pass  # Question 4